In [8]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import mean_squared_error, accuracy_score, classification_report
from sklearn.model_selection import RandomizedSearchCV, train_test_split
import logging
import numpy as np

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

def load_data(file_path):
    try:
        data = pd.read_csv(file_path)
        logging.info(f"Dataset {file_path} loaded successfully.")
        return data
    except Exception as e:
        logging.error(f"Error loading dataset {file_path}: {e}")
        return None

def preprocess_data(data, age_column_name):
    data = data.copy()

    # Convert categorical variables to numerical using LabelEncoder
    label_encoders = {}
    categorical_columns = ['name', 'based_in', 'birth_city', 'nation_of_birth', 'birth_region', 'nationality', 
                           'position', 'group', 'club', 'division', 'division_tier', 'second_nationality', 
                           'is_top_4_tier']

    for col in categorical_columns:
        if col in data.columns:
            le = LabelEncoder()
            data[col] = le.fit_transform(data[col].astype(str))
            label_encoders[col] = le
        else:
            logging.warning(f"Column '{col}' not found in the dataset.")

    # Handle missing values - fill NaNs with a placeholder (e.g., -1) or the median for numerical columns
    data.fillna(-1, inplace=True)

    return data, label_encoders

def prepare_features_and_targets(data, age_column_name):
    features = data[['position', 'group', age_column_name, 'birth_month', 'birth_quarter', 'second_nationality', 
                     'height_(cm)', 'weight_(kg)', 'club', 'division', 'division_tier', 'tier_quality']]
    target_goals = data['goals'].astype(int)
    target_appearances = data['appearances'].astype(int)
    target_tier_quality = data['tier_quality']

    X = features
    y_goals = target_goals
    y_appearances = target_appearances
    y_tier_quality = target_tier_quality

    return X, y_goals, y_appearances, y_tier_quality

def tune_hyperparameters(X_train, y_train, is_classifier=False):
    param_distributions = {
        'n_estimators': np.arange(100, 1100, 100),
        'max_depth': [None] + list(np.arange(10, 51, 10)),
        'min_samples_split': [2, 5, 10, 15, 20],
        'min_samples_leaf': [1, 2, 5, 10]
    }
    
    if is_classifier:
        model = RandomForestClassifier(random_state=42)
        scoring = 'accuracy'
    else:
        model = RandomForestRegressor(random_state=42)
        scoring = 'neg_mean_squared_error'
    
    random_search = RandomizedSearchCV(estimator=model, param_distributions=param_distributions, 
                                       n_iter=100, cv=5, n_jobs=-1, verbose=2, scoring=scoring, random_state=42)
    
    random_search.fit(X_train, y_train)
    logging.info(f"Best parameters: {random_search.best_params_}")
    return random_search.best_estimator_

def train_and_evaluate(X_train, y_train_goals, y_train_appearances, y_train_tier_quality, X_val, y_val_goals, y_val_appearances, y_val_tier_quality):
    # Tune hyperparameters for goals prediction
    best_rf_goals = tune_hyperparameters(X_train, y_train_goals)
    best_rf_goals.fit(X_train, y_train_goals)

    # Tune hyperparameters for appearances prediction
    best_rf_appearances = tune_hyperparameters(X_train, y_train_appearances)
    best_rf_appearances.fit(X_train, y_train_appearances)

    # Tune hyperparameters for tier quality prediction
    best_rf_tier_quality = tune_hyperparameters(X_train, y_train_tier_quality, is_classifier=True)
    best_rf_tier_quality.fit(X_train, y_train_tier_quality)

    # Evaluate the models on validation set
    y_val_pred_goals = best_rf_goals.predict(X_val)
    y_val_pred_appearances = best_rf_appearances.predict(X_val)
    y_val_pred_tier_quality = best_rf_tier_quality.predict(X_val)

    logging.info(f'Validation MSE Goals: {mean_squared_error(y_val_goals, y_val_pred_goals):.0f}')
    logging.info(f'Validation MSE Appearances: {mean_squared_error(y_val_appearances, y_val_pred_appearances):.0f}')
    logging.info(f'Validation Accuracy Tier Quality: {accuracy_score(y_val_tier_quality, y_val_pred_tier_quality)}')
    logging.info(f'Validation Classification Report Tier Quality:\n{classification_report(y_val_tier_quality, y_val_pred_tier_quality)}')

    return best_rf_goals, best_rf_appearances, best_rf_tier_quality

def evaluate_models(best_rf_goals, best_rf_appearances, best_rf_tier_quality, X_test, y_test_goals, y_test_appearances, y_test_tier_quality):
    # Make predictions on the testing set
    y_pred_goals = best_rf_goals.predict(X_test)
    y_pred_appearances = best_rf_appearances.predict(X_test)
    y_pred_tier_quality = best_rf_tier_quality.predict(X_test)

    # Evaluate the models
    mse_goals = mean_squared_error(y_test_goals, y_pred_goals)
    mse_appearances = mean_squared_error(y_test_appearances, y_pred_appearances)
    accuracy_tier_quality = accuracy_score(y_test_tier_quality, y_pred_tier_quality)

    logging.info(f'MSE Goals: {mse_goals:.0f}')
    logging.info(f'MSE Appearances: {mse_appearances:.0f}')
    logging.info(f'Accuracy Tier Quality: {accuracy_tier_quality}')
    logging.info(f'Test Classification Report Tier Quality:\n{classification_report(y_test_tier_quality, y_pred_tier_quality)}')

    # Convert feature importances to percentages
    feature_names = X_test.columns  # Get feature names
    goals_feature_importances = best_rf_goals.feature_importances_ * 100
    appearances_feature_importances = best_rf_appearances.feature_importances_ * 100
    tier_quality_feature_importances = best_rf_tier_quality.feature_importances_ * 100

    # Pair feature names with their importance percentages, formatted to 2 decimal places
    goals_importances_with_names = {name: f'{imp:.2f}' for name, imp in zip(feature_names, goals_feature_importances)}
    appearances_importances_with_names = {name: f'{imp:.2f}' for name, imp in zip(feature_names, appearances_feature_importances)}
    tier_quality_importances_with_names = {name: f'{imp:.2f}' for name, imp in zip(feature_names, tier_quality_feature_importances)}

    # Log the feature importances with names
    logging.info(f'Feature importances for goals (as percentages): {goals_importances_with_names}')
    logging.info(f'Feature importances for appearances (as percentages): {appearances_importances_with_names}')
    logging.info(f'Feature importances for tier quality (as percentages): {tier_quality_importances_with_names}')

    return y_pred_goals, y_pred_appearances, y_pred_tier_quality

def save_predictions(original_data, X_test, y_pred_goals, y_pred_appearances, y_pred_tier_quality, output_file):
    # Filter the original data to match the X_test indices
    filtered_original_data = original_data.loc[X_test.index]

    # Create predictions DataFrame
    predictions = pd.DataFrame({
        'name': filtered_original_data['name'],
        'position': filtered_original_data['position'],
        'group': filtered_original_data['group'],
        'date_of_birth': filtered_original_data['date_of_birth'],
        'birth_month': filtered_original_data['birth_month'],
        'birth_quarter': filtered_original_data['birth_quarter'],  
        'appearances_pred': np.round(y_pred_appearances, 0).astype(int),
        'goals_pred': np.round(y_pred_goals, 0).astype(int),
        'tier_pred': y_pred_tier_quality
    })
    
    # Save predictions to CSV
    predictions.to_csv(output_file, index=False)
    logging.info(f"Predictions have been saved to '{output_file}'.")

def main():
    # Load and preprocess the 2016 senior players dataset
    data_senior_2016 = load_data('dataset_2016_senior_players.csv')
    if data_senior_2016 is not None:
        logging.info(f"Columns in the 2016 senior dataset: {data_senior_2016.columns.tolist()}")
        data_senior_preprocessed_2016, _ = preprocess_data(data_senior_2016, 'age_(months)_on_1_july_2015')

        # Filter the data for players aged 23 or below (276 months or below) and not goalkeepers
        data_senior_preprocessed_2016 = data_senior_preprocessed_2016[
            (data_senior_preprocessed_2016['age_(months)_on_1_july_2015'] <= 276) &
            (data_senior_preprocessed_2016['group'] != 2)  # Where '2' is the encoded value for goalkeepers
        ]

        X_senior_2016, y_goals_senior_2016, y_appearances_senior_2016, y_tier_quality_senior_2016 = prepare_features_and_targets(data_senior_preprocessed_2016, 'age_(months)_on_1_july_2015')
        
        # Split the senior dataset into training and validation sets
        X_train_senior_2016, X_val_senior_2016, y_train_goals_senior_2016, y_val_goals_senior_2016, y_train_appearances_senior_2016, y_val_appearances_senior_2016, y_train_tier_quality_senior_2016, y_val_tier_quality_senior_2016 = train_test_split(
            X_senior_2016, y_goals_senior_2016, y_appearances_senior_2016, y_tier_quality_senior_2016, test_size=0.2, random_state=42)
        
        # Train the models on the senior dataset and evaluate on the validation set
        best_rf_goals_2016, best_rf_appearances_2016, best_rf_tier_quality_2016 = train_and_evaluate(
            X_train_senior_2016, y_train_goals_senior_2016, y_train_appearances_senior_2016, y_train_tier_quality_senior_2016,
            X_val_senior_2016, y_val_goals_senior_2016, y_val_appearances_senior_2016, y_val_tier_quality_senior_2016)

    # Load and preprocess the 2016 youth players dataset
    data_youth_2016 = load_data('dataset_2016_youth_players.csv')
    if data_youth_2016 is not None:
        logging.info(f"Columns in the 2016 youth dataset: {data_youth_2016.columns.tolist()}")
        data_youth_preprocessed_2016, _ = preprocess_data(data_youth_2016, 'age_(months)_on_1_july_2015')

        # Filter the data for not goalkeepers
        data_youth_preprocessed_2016 = data_youth_preprocessed_2016[
            (data_youth_preprocessed_2016['group'] != 2)  # Where '2' is the encoded value for goalkeepers
        ]

        X_test_youth_2016, y_test_goals_youth_2016, y_test_appearances_youth_2016, y_test_tier_quality_youth_2016 = prepare_features_and_targets(data_youth_preprocessed_2016, 'age_(months)_on_1_july_2015')
        
        # Make predictions on the 2016 youth dataset using the models trained on the 2016 senior dataset
        y_pred_goals_youth_2016, y_pred_appearances_youth_2016, y_pred_tier_quality_youth_2016 = evaluate_models(
            best_rf_goals_2016, best_rf_appearances_2016, best_rf_tier_quality_2016, X_test_youth_2016, y_test_goals_youth_2016, y_test_appearances_youth_2016, y_test_tier_quality_youth_2016)

        # Sorting and logging feature importances for the 2016 youth predictions
        sorted_goals_importances_2016 = dict(sorted(zip(X_test_youth_2016.columns, best_rf_goals_2016.feature_importances_ * 100), key=lambda item: item[1], reverse=True))
        sorted_appearances_importances_2016 = dict(sorted(zip(X_test_youth_2016.columns, best_rf_appearances_2016.feature_importances_ * 100), key=lambda item: item[1], reverse=True))
        sorted_tier_quality_importances_2016 = dict(sorted(zip(X_test_youth_2016.columns, best_rf_tier_quality_2016.feature_importances_ * 100), key=lambda item: item[1], reverse=True))

        logging.info(f'Feature importances for goals (2016 youth) (as percentages): {sorted_goals_importances_2016}')
        logging.info(f'Feature importances for appearances (2016 youth) (as percentages): {sorted_appearances_importances_2016}')
        logging.info(f'Feature importances for tier quality (2016 youth) (as percentages): {sorted_tier_quality_importances_2016}')

        # Save predictions for the 2016 youth dataset
        save_predictions(data_youth_2016, X_test_youth_2016, y_pred_goals_youth_2016, y_pred_appearances_youth_2016, y_pred_tier_quality_youth_2016, 'predictions_2016_youth.csv')

    # Load and preprocess the 2024 senior players dataset
    data_senior_2024 = load_data('dataset_2024_senior_players.csv')
    if data_senior_2024 is not None:
        logging.info(f"Columns in the 2024 senior dataset: {data_senior_2024.columns.tolist()}")
        data_senior_preprocessed_2024, _ = preprocess_data(data_senior_2024, 'age_(months)_on_1_july_2023')

        # Filter the data for players aged 23 or below (276 months or below) and not goalkeepers
        data_senior_preprocessed_2024 = data_senior_preprocessed_2024[
            (data_senior_preprocessed_2024['age_(months)_on_1_july_2023'] <= 276) &
            (data_senior_preprocessed_2024['group'] != 2)  # Where '2' is the encoded value for goalkeepers
        ]

        X_senior_2024, y_goals_senior_2024, y_appearances_senior_2024, y_tier_quality_senior_2024 = prepare_features_and_targets(data_senior_preprocessed_2024, 'age_(months)_on_1_july_2023')
        
        # Split the senior dataset into training and validation sets
        X_train_senior_2024, X_val_senior_2024, y_train_goals_senior_2024, y_val_goals_senior_2024, y_train_appearances_senior_2024, y_val_appearances_senior_2024, y_train_tier_quality_senior_2024, y_val_tier_quality_senior_2024 = train_test_split(
            X_senior_2024, y_goals_senior_2024, y_appearances_senior_2024, y_tier_quality_senior_2024, test_size=0.2, random_state=42)
        
        # Train the models on the senior dataset and evaluate on the validation set
        best_rf_goals_2024, best_rf_appearances_2024, best_rf_tier_quality_2024 = train_and_evaluate(
            X_train_senior_2024, y_train_goals_senior_2024, y_train_appearances_senior_2024, y_train_tier_quality_senior_2024,
            X_val_senior_2024, y_val_goals_senior_2024, y_val_appearances_senior_2024, y_val_tier_quality_senior_2024)

    # Load and preprocess the 2024 youth players dataset
    data_youth_2024 = load_data('dataset_2024_youth_players.csv')
    if data_youth_2024 is not None:
        logging.info(f"Columns in the 2024 youth dataset: {data_youth_2024.columns.tolist()}")
        data_youth_preprocessed_2024, _ = preprocess_data(data_youth_2024, 'age_(months)_on_1_july_2023')

        # Filter the data for not goalkeepers
        data_youth_preprocessed_2024 = data_youth_preprocessed_2024[
            (data_youth_preprocessed_2024['group'] != 2)  # Where '2' is the encoded value for goalkeepers
        ]

        X_test_youth_2024, y_test_goals_youth_2024, y_test_appearances_youth_2024, y_test_tier_quality_youth_2024 = prepare_features_and_targets(data_youth_preprocessed_2024, 'age_(months)_on_1_july_2023')
        
        # Make predictions on the 2024 youth dataset using the models trained on the 2024 senior dataset
        y_pred_goals_youth_2024, y_pred_appearances_youth_2024, y_pred_tier_quality_youth_2024 = evaluate_models(
            best_rf_goals_2024, best_rf_appearances_2024, best_rf_tier_quality_2024, X_test_youth_2024, y_test_goals_youth_2024, y_test_appearances_youth_2024, y_test_tier_quality_youth_2024)

        # Sorting and logging feature importances for the 2024 youth predictions
        sorted_goals_importances_2024 = dict(sorted(zip(X_test_youth_2024.columns, best_rf_goals_2024.feature_importances_ * 100), key=lambda item: item[1], reverse=True))
        sorted_appearances_importances_2024 = dict(sorted(zip(X_test_youth_2024.columns, best_rf_appearances_2024.feature_importances_ * 100), key=lambda item: item[1], reverse=True))
        sorted_tier_quality_importances_2024 = dict(sorted(zip(X_test_youth_2024.columns, best_rf_tier_quality_2024.feature_importances_ * 100), key=lambda item: item[1], reverse=True))

        logging.info(f'Feature importances for goals (2024 youth) (as percentages): {sorted_goals_importances_2024}')
        logging.info(f'Feature importances for appearances (2024 youth) (as percentages): {sorted_appearances_importances_2024}')
        logging.info(f'Feature importances for tier quality (2024 youth) (as percentages): {sorted_tier_quality_importances_2024}')

        # Save predictions for the 2024 youth dataset
        save_predictions(data_youth_2024, X_test_youth_2024, y_pred_goals_youth_2024, y_pred_appearances_youth_2024, y_pred_tier_quality_youth_2024, 'predictions_2024_youth.csv')

if __name__ == "__main__":
    main()


2024-08-09 17:28:33,155 - INFO - Dataset dataset_2016_senior_players.csv loaded successfully.
2024-08-09 17:28:33,155 - INFO - Columns in the 2016 senior dataset: ['name', 'position', 'group', 'club', 'division', 'based_in', 'division_tier', 'tier_quality', 'date_of_birth', 'birth_month', 'birth_quarter', 'age_(days)_on_1_july_2015', 'age_(months)_on_1_july_2015', 'age_(years)_on_1_july_2015', 'birth_city', 'nation_of_birth', 'birth_region', 'nationality', 'second_nationality', 'height_(ft_in)', 'height_(ft)', 'height_(in)', 'height_(cm)', 'weight_(kg)', 'goals', 'appearances', 'is_top_4_tier', 'train_or_test']


Fitting 5 folds for each of 100 candidates, totalling 500 fits
[CV] END max_depth=40, min_samples_leaf=2, min_samples_split=5, n_estimators=600; total time=   1.0s
[CV] END max_depth=50, min_samples_leaf=10, min_samples_split=10, n_estimators=900; total time=   1.2s
[CV] END max_depth=50, min_samples_leaf=10, min_samples_split=10, n_estimators=900; total time=   1.1s
[CV] END max_depth=40, min_samples_leaf=2, min_samples_split=5, n_estimators=600; total time=   1.1s
[CV] END max_depth=50, min_samples_leaf=10, min_samples_split=10, n_estimators=900; total time=   1.2s
[CV] END max_depth=50, min_samples_leaf=10, min_samples_split=10, n_estimators=900; total time=   1.2s
[CV] END max_depth=50, min_samples_leaf=10, min_samples_split=10, n_estimators=900; total time=   1.2s
[CV] END max_depth=40, min_samples_leaf=2, min_samples_split=5, n_estimators=600; total time=   1.2s
[CV] END max_depth=None, min_samples_leaf=5, min_samples_split=2, n_estimators=200; total time=   0.2s
[CV] END max_dep

2024-08-09 17:29:29,444 - INFO - Best parameters: {'n_estimators': 1000, 'min_samples_split': 20, 'min_samples_leaf': 1, 'max_depth': None}


Fitting 5 folds for each of 100 candidates, totalling 500 fits
[CV] END max_depth=50, min_samples_leaf=10, min_samples_split=10, n_estimators=900; total time=   1.2s
[CV] END max_depth=40, min_samples_leaf=2, min_samples_split=5, n_estimators=600; total time=   1.1s
[CV] END max_depth=40, min_samples_leaf=2, min_samples_split=5, n_estimators=600; total time=   1.1s
[CV] END max_depth=40, min_samples_leaf=2, min_samples_split=5, n_estimators=600; total time=   1.2s
[CV] END max_depth=50, min_samples_leaf=10, min_samples_split=10, n_estimators=900; total time=   1.3s
[CV] END max_depth=50, min_samples_leaf=10, min_samples_split=10, n_estimators=900; total time=   1.3s
[CV] END max_depth=50, min_samples_leaf=10, min_samples_split=10, n_estimators=900; total time=   1.3s
[CV] END max_depth=50, min_samples_leaf=10, min_samples_split=10, n_estimators=900; total time=   1.3s
[CV] END max_depth=None, min_samples_leaf=5, min_samples_split=2, n_estimators=200; total time=   0.3s
[CV] END max_dep

2024-08-09 17:30:42,957 - INFO - Best parameters: {'n_estimators': 100, 'min_samples_split': 10, 'min_samples_leaf': 10, 'max_depth': None}
/Applications/miniconda3/lib/python3.12/site-packages/sklearn/model_selection/_split.py:776: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


...

2024-08-09 17:34:37,125 - INFO - Best parameters: {'n_estimators': 500, 'min_samples_split': 2, 'min_samples_leaf': 10, 'max_depth': 10}


Fitting 5 folds for each of 100 candidates, totalling 500 fits


/Applications/miniconda3/lib/python3.12/site-packages/sklearn/model_selection/_split.py:776: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


[CV] END max_depth=40, min_samples_leaf=2, min_samples_split=5, n_estimators=600; total time=   1.2s
[CV] END max_depth=40, min_samples_leaf=2, min_samples_split=5, n_estimators=600; total time=   1.2s
[CV] END max_depth=40, min_samples_leaf=2, min_samples_split=5, n_estimators=600; total time=   1.5s
[CV] END max_depth=50, min_samples_leaf=10, min_samples_split=10, n_estimators=900; total time=   1.7s
[CV] END max_depth=50, min_samples_leaf=10, min_samples_split=10, n_estimators=900; total time=   1.8s
[CV] END max_depth=50, min_samples_leaf=10, min_samples_split=10, n_estimators=900; total time=   1.8s
[CV] END max_depth=50, min_samples_leaf=10, min_samples_split=10, n_estimators=900; total time=   1.8s
[CV] END max_depth=50, min_samples_leaf=10, min_samples_split=10, n_estimators=900; total time=   1.8s
[CV] END max_depth=None, min_samples_leaf=5, min_samples_split=2, n_estimators=200; total time=   0.5s
[CV] END max_depth=None, min_samples_leaf=5, min_samples_split=2, n_estimators=

2024-08-09 17:35:40,896 - INFO - Best parameters: {'n_estimators': 200, 'min_samples_split': 5, 'min_samples_leaf': 1, 'max_depth': 20}
2024-08-09 17:35:41,073 - INFO - Validation MSE Goals: 47
2024-08-09 17:35:41,073 - INFO - Validation MSE Appearances: 642
2024-08-09 17:35:41,074 - INFO - Validation Accuracy Tier Quality: 0.9904761904761905
/Applications/miniconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Applications/miniconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(re